Ground Truth Image Set Capture Conditions 
- 24 photos of the same Post-it note was taken using UTC's imaging aparatus.
- A metal washer was used to keep the non-adhesive side of the post-it flush.
- Images were taken at varying focal distances.
- Distance measurements are according to kiser Copy Stand's arm measurements and are not accurate to actual  focal distances due to the photo box's stage height offset.
    - ranged from 82cm - 61cm
    - photo box height offset is constant and makes focal distance closer by (need to take measurement) XXcm.
    - it was not possible to retain the color reference in images closer than 61cm.

- Post-it notes are 7.62 cm x 7.62 cm

In [1]:
import numpy as np
from math import sqrt
import cv2
from glob import glob
from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow
plt.rcParams['figure.figsize'] = [25, 25]

import piexif
import json

#### Identify the image test set

In [2]:
imgs = glob("./scale_test_set/processed/*.jpg")
len(imgs)

24

#### Extract the scale values for each image as determined by HerbASAP

In [60]:
def cvt_num(x):
    try:
        x = float(x)
    except:
        x = np.nan
    return x

def cvt_ci(scale, ci):
    scale = cvt_num(scale)
    ci = cvt_num(ci)
    try:
        if (ci == scale) or (ci > 50):
            ci = np.nan
            scale = np.nan
        else:
            ci = float(ci)
            scale = float(scale)
    except:
        ci = np.nan
        scale = np.nan
    return scale, ci

def extract_id(path):
    img_id = path.split("/")[-1].split(".jpg")[0]
    return img_id

def remove_vignette(img, correction_factor=0.75):
    corrections = cv2.imread("./scale_test_set/vignette_correction.jpg")
    corrections = (corrections * correction_factor) // 255
    corrections = corrections.astype('uint8')
    img = (img + corrections)
    img = cv2.normalize(img,  img, 0, 255, cv2.NORM_MINMAX)

    # verify results are as expected
    #cv2.imwrite("corrected.jpg", img)
    return img

def scale_contour(cnt, scale):
    # taken from:
    # https://medium.com/analytics-vidhya/tutorial-how-to-scale-and-rotate-contours-in-opencv-using-python-f48be59c35a2
    M = cv2.moments(cnt)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])

    cnt_norm = cnt - [cx, cy]
    cnt_scaled = cnt_norm * scale
    cnt_scaled = cnt_scaled + [cx, cy]
    cnt_scaled = cnt_scaled.astype(np.int32)

    return cnt_scaled

def postit_pxs(imgpath):
    """adapted from: https://stackoverflow.com/questions/44588279/find-and-draw-the-largest-contour-in-opencv-on-a-specific-color-python"""
    im = cv2.imread(imgpath)
    im = remove_vignette(im)

    # crop 20% of image's height to remove crc and 10% of image's width to remove guides
    l, w, _ = im.shape
    im = im[l//10:l - l//10, w//20: w - w//20, ...]
    image = im
    # establish color boundries for the post-it
    lower = [150, 150, 150]
    upper = [255, 255, 255]

    # create NumPy arrays from the boundaries
    lower = np.array(lower, dtype="uint8")
    upper = np.array(upper, dtype="uint8")

    # find the colors within the specified boundaries and apply
    mask = cv2.inRange(image, lower, upper)
    output = cv2.bitwise_and(image, image, mask=mask)

    cont_image = cv2.cvtColor(output, cv2.COLOR_BGR2GRAY)
    contours, hierarchy = cv2.findContours(cont_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    if len(contours) != 0:
        contour = max(contours, key = cv2.contourArea)
        # generate dummy image to paint the contour onto
        contour_mask = np.zeros(cont_image.shape, np.uint8)
        cv2.drawContours(contour_mask, [contour], -1, (255,255,255), -1)
        
        area = np.sqrt(cv2.countNonZero(contour_mask))
        #area = cv2.contourArea(contour)

        # draw the identified contour in red
        #cv2.drawContours(output, contour, -1, (50, 50, 255) , 5)
        #cv2.imwrite(f'{extract_id(imgpath)}_contour.jpg',contour_mask)
    else:
        area = False
    return area

results = {}
#for imgpath in ["./scale_test_set/processed/59.jpg"]:
for imgpath in set(imgs):
    img_id = extract_id(imgpath)
    exifDict = piexif.load(imgpath)
    desc = json.loads(exifDict["0th"][270])
    pixelsPerMM, CI = cvt_ci(desc["pixelsPerMM"], desc["pixelsPerMMConfidence"])

    postit_area_px = postit_pxs(imgpath)
    p_area_mm = round(postit_area_px / pixelsPerMM, 3)
    p_area_ci_min = round(postit_area_px / (pixelsPerMM + CI), 3)
    p_area_ci_max = round(postit_area_px / (pixelsPerMM - CI), 3)

    # results organized as list under key. 
    
    results[img_id] = [pixelsPerMM, CI, postit_area_px, p_area_mm, p_area_ci_min, p_area_ci_max]

#### Desired output ~ 76.2mm^2


post-it notes are 3" x 3" (7.62 cm x 7.62 cm)

In [61]:
# for #74, gimp px count is 1089051
ids = list(results.keys())
ids.sort()

areas = []
deviations = []
deviation_pcts = []
is_contained_in_ci = []
for id in ids:
    print(f"{id}: {results.get(id)}")
    pixelsPerMM, CI, postit_area_px, p_area_mm, p_area_ci_min, p_area_ci_max = results.get(id)

    deviation = p_area_mm - 76.2
    deviations.append(deviation)
    deviation_pct = (abs(deviation) / 76.2) * 100
    deviation_pcts.append(deviation_pct)
    
    # check if CI contains expected value
    if (p_area_ci_min <= 76.2 <= p_area_ci_max):
        is_contained = True
    else:
        is_contained = False
    is_contained_in_ci.append(is_contained)
    print(is_contained)
    
    areas.append(p_area_mm)
# List order results are:[pixelsPerMM, CI, postit_area_px, p_area_mm, p_area_ci_min, p_area_ci_max]

avg_area = np.mean(areas)
avg_area_diff = avg_area - 76.2
pct_contained = np.round(is_contained_in_ci.count(True) / len(is_contained_in_ci) *100, 3)

print()
print(f"mean difference from 76.2 = {avg_area_diff}")
print(f"deviation pcts (either direction) = {np.round(np.mean(deviation_pcts), 3)}%")
print(f"pct of results containing 76.2  in ci = {pct_contained}%")

59: [18.481, 0.341, 1392.069322986467, 75.324, 73.96, 76.74]
True
60: [17.939, 0.303, 1362.8151011784394, 75.969, 74.708, 77.275]
True
61: [17.803, 0.304, 1332.6815073377434, 74.857, 73.6, 76.158]
False
62: [17.237, 0.244, 1304.9697314497375, 75.707, 74.651, 76.795]
True
63: [17.033, 0.363, 1279.4811448395792, 75.118, 73.55, 76.754]
True
64: [16.634, 0.203, 1254.9513934810384, 75.445, 74.535, 76.377]
True
65: [16.17, 0.206, 1228.2650365454517, 75.959, 75.004, 76.94]
True
66: [15.994, 0.283, 1204.8103585212073, 75.329, 74.019, 76.686]
True
67: [15.612, 0.232, 1182.5303378772148, 75.745, 74.636, 76.888]
True
68: [15.161, 0.185, 1160.4210442766023, 76.54, 75.617, 77.485]
True
69: [14.974, 0.27, 1139.4718074616853, 76.097, 74.749, 77.494]
True
70: [14.071, 0.849, 1118.9052685549389, 79.519, 74.994, 84.625]
True
71: [14.395, 0.225, 1098.8730590928144, 76.337, 75.162, 77.549]
True
72: [14.109, 0.223, 1080.0911998530494, 76.553, 75.362, 77.783]
True
73: [13.85, 0.221, 1060.9571150616787, 76.6

In [63]:
deviations

[-0.8760000000000048,
 -0.23100000000000875,
 -1.3430000000000035,
 -0.4930000000000092,
 -1.0820000000000078,
 -0.7550000000000097,
 -0.24099999999999966,
 -0.8710000000000093,
 -0.4549999999999983,
 0.3400000000000034,
 -0.10300000000000864,
 3.3190000000000026,
 0.13700000000000045,
 0.35299999999999443,
 0.4029999999999916,
 0.19499999999999318,
 0.28300000000000125,
 0.1039999999999992,
 0.48699999999999477,
 3.6499999999999915,
 1.0180000000000007,
 0.840999999999994,
 0.9050000000000011,
 0.8310000000000031]

old results for holding (sigmoid)

In [51]:
# for #74, gimp px count is 1089051
ids = list(results.keys())
ids.sort()

areas = []
deviations = []
deviation_pcts = []
is_contained_in_ci = []
for id in ids:
    print(f"{id}: {results.get(id)}")
    pixelsPerMM, CI, postit_area_px, p_area_mm, p_area_ci_min, p_area_ci_max = results.get(id)

    deviation = p_area_mm - 76.2
    deviations.append(deviation)
    deviation_pct = (abs(deviation) / 76.2) * 100
    deviation_pcts.append(deviation_pct)
    
    # check if CI contains expected value
    if (p_area_ci_min <= 76.2 <= p_area_ci_max):
        is_contained = True
    else:
        is_contained = False
    is_contained_in_ci.append(is_contained)
    print(is_contained)
    
    areas.append(p_area_mm)
# List order results are:[pixelsPerMM, CI, postit_area_px, p_area_mm, p_area_ci_min, p_area_ci_max]

avg_area = np.mean(areas)
avg_area_diff = avg_area - 76.2
pct_contained = np.round(is_contained_in_ci.count(True) / len(is_contained_in_ci) *100, 3)

print()
print(f"mean difference from 76.2 = {avg_area_diff}")
print(f"deviation pcts (either direction) = {np.round(np.mean(deviation_pcts), 3)}%")
print(f"pct of results containing 76.2  in ci = {pct_contained}%")

59: [18.445, 0.341, 1392.069322986467, 75.471, 74.101, 76.893]
True
60: [17.902, 0.304, 1362.8151011784394, 76.126, 74.855, 77.441]
True
61: [17.767, 0.306, 1332.6188502343796, 75.005, 73.735, 76.32]
True
62: [17.218, 0.236, 1304.9697314497375, 75.791, 74.766, 76.844]
True
63: [17.011, 0.33, 1279.4811448395792, 75.215, 73.784, 76.703]
True
64: [16.556, 0.198, 1254.9513934810384, 75.8, 74.905, 76.718]
True
65: [16.132, 0.206, 1228.2788771284802, 76.139, 75.179, 77.124]
True
66: [15.949, 0.254, 1204.8103585212073, 75.541, 74.357, 76.764]
True
67: [15.573, 0.233, 1182.5303378772148, 75.935, 74.815, 77.088]
True
68: [15.168, 0.212, 1160.4210442766023, 76.505, 75.45, 77.589]
True
69: [14.935, 0.27, 1139.4718074616853, 76.295, 74.941, 77.7]
True
70: [14.625, 0.229, 1118.9052685549389, 76.506, 75.327, 77.723]
True
71: [14.355, 0.225, 1098.8730590928144, 76.55, 75.369, 77.769]
True
72: [14.07, 0.223, 1080.0911998530494, 76.766, 75.568, 78.002]
True
73: [13.811, 0.221, 1060.9571150616787, 76.82

In [52]:
deviations

[-0.7289999999999992,
 -0.07399999999999807,
 -1.1950000000000074,
 -0.409000000000006,
 -0.9849999999999994,
 -0.4000000000000057,
 -0.06100000000000705,
 -0.659000000000006,
 -0.26500000000000057,
 0.3049999999999926,
 0.09499999999999886,
 0.3059999999999974,
 0.3499999999999943,
 0.5660000000000025,
 0.6199999999999903,
 0.4140000000000015,
 0.6670000000000016,
 0.3359999999999985,
 0.7180000000000035,
 1.7959999999999923,
 1.6039999999999992,
 1.1679999999999922,
 2.3569999999999993,
 1.0900000000000034]